Koristimo **MovieLens** dataset. Podaci su podeljeni u dva skupa. Prvi skup koji koristimo sadrži listu filmova sa njihovim prosečnim ocenama i atributima poput budžeta, prihoda, uloga itd. Najpre analiziramo podatke.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
dataset1 = pd.read_csv("the-movies-dataset/movies_metadata.csv")

In [4]:
type(dataset1)

pandas.core.frame.DataFrame

In [7]:
dataset1.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Eliminišemo duplikate iz podataka.

In [8]:
print('Number of entries: ',len(dataset1))
dataset1 = dataset1.drop_duplicates()
print('Number of different entries: ',len(dataset1))

Number of entries:  45453
Number of different entries:  45453


Pravimo listu TopRated koja sadrži najbolje ocenjene filmove prema sistemu *IMDB weighting*. Računamo otežanu ocenu za svaki film prema formuli
$ \frac{v}{v+m} * R + \frac{m}{v+m}*C$, gde je $R$ prosečna ocena filma (atribut *vote_average*), $v$ broj glasova za film (atribut *vote_count*), $m$ minimalan broj glasova neophodan da bi se film našao na TopRated listi (**postavljamo na 90% glasova -- nejasan deo** što je negde oko 160) i $C$ je srednja ocena uzeta po svim dostupnim filmovima. Prikazujemo 10 najpopularnijih filmova po ovom kriterijumu.

In [9]:
v = dataset1.vote_count
m = 159.994 
R = dataset1.vote_average
C = dataset1.vote_average.mean()
weighted_score = (v/(v+m))*R + (m/(v+m))*C
weighted_score = round(weighted_score, 6)

movie_title = dataset1.title
top_rated = np.vstack([movie_title,R,v,weighted_score]).T
top_rated = pd.DataFrame(top_rated,columns = ['Movie Title','Avg Votes','Num Votes','Weighted Score']).sort_values('Weighted Score',ascending=False)

top_rated.head(10)

,Movie Title,Avg Votes,Num Votes,Weighted Score
314,The Shawshank Redemption,8.5,8358,8.44587
834,The Godfather,8.5,6024,8.42544
10306,Dilwale Dulhania Le Jayenge,9.1,661,8.42147
12478,The Dark Knight,8.3,12269,8.26548
2842,Fight Club,8.3,9678,8.25639
292,Pulp Fiction,8.3,8670,8.25141
522,Schindler's List,8.3,4436,8.20664
23664,Whiplash,8.3,4376,8.20541
5480,Spirited Away,8.3,3968,8.19606
2210,Life Is Beautiful,8.3,3643,8.18717


Delimo skup podataka *dataset1* na skupove za treniranje i testiranje, u razmeri 4:1

In [10]:
from sklearn import model_selection

train1, test1 = model_selection.train_test_split(dataset1, test_size = 0.2)
print(len(train1),len(test1))

36362 9091


Drugi skup podataka koji koristimo sastoji se od ID-a korisnika, ID-a filma i ocene od 1 do 5 koju je korisnik dao filmu. Ove podatke predstavljamo kao matricu tako da jedna dimenzija odgovara korisnicima a druga filmovima. Ova matrica je dosta retka zato što je većina korisnika ocenila samo mali udeo filmova. Koristimo manji skup **ratings_small.csv** umesto ogromnog **ratings.csv** zbog računarskih ograničenja.

In [11]:
dataset2 = pd.read_csv("the-movies-dataset/ratings_small.csv")
dataset2.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [12]:
number_of_users = dataset2.userId.unique().shape[0]
print('Broj razlicitih korisnika:', number_of_users)

number_of_movies = dataset2.movieId.unique().shape[0]
print('Broj razlicitih filmova:', number_of_movies)
max(dataset2.movieId)

Broj razlicitih korisnika: 671
Broj razlicitih filmova: 9066


163949

In [13]:
print('Maksimalan movieId: ',max(dataset2.movieId))

Maksimalan movieId:  163949


Iako ima 9066 razlicitih *movieId*-eva u skupu, pravimo matricu koja će imati onoliko kolona koliki je maksimalan *movieId*, što je 163 949. Jasno, neke kolone će biti nula kolone što znači da za taj film među podacima nema ni jedne ocene od strane korisnika. Razlog za ovo je što smo koristili nasumičan podskup 100 004 instanci, naspram ukupnog skupa podataka koji ima preko 25 miliona. Kako je u pitanju retka matrica, koristimo funkciju *dok_matrix* iz paketa **numpy.sparse**.

In [14]:
number_of_movies = max(dataset2.movieId)

In [15]:
from scipy import sparse as sps
data_matrix = sps.dok_matrix((number_of_users,number_of_movies),dtype=np.float64)
data_matrix

<671x163949 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Dictionary Of Keys format>

In [16]:
for row in dataset2.itertuples():
    data_matrix[row[1]-1,row[2]-1] = row[3]

Ocena koju je korisnik sa id-om $1$ dao filmu sa id-om $1029$ je $3.0$. Proverimo da li se slaže to sa našom matricom podataka.

In [17]:
print(data_matrix[0,1028])

3.0


Delimo skup podataka *dataset2* na skupove za treniranje i testiranje, u razmeri 4:1

In [18]:
train2, test2 = model_selection.train_test_split(dataset2, test_size = 0.2)
print(len(train2),len(test2))

80003 20001


Za content-based pristup izgradnji sistema za preporuku koristimo metriku **TF-IDF**, pomoću koje se terminu dodeljuje težina. Važnost termina (reči) $t$ raste sa brojem njegovog pojavljivanja u dokumentu $d$, ali se otezava njegovim ukupnim brojem pojavljivanja u celokupnom skupu dokumenata (koji ima više dokumenata). Za reč $t$ u dokumentu $d$ računamo meru **tf-idf(t)** kao proizvod sledeće dve veličine:
* $TF(t, d)$ - Broj pojavljivanja reči $t$ u dokumentu $d$
*    $IDF(t, d, D)$ - Logaritam količnika ukupnog broja dokumenata $d$ sa brojem dokumenata (iz skupa $D$) u kojima se pojavljuje reč $t$


In [19]:
from sklearn import feature_extraction

In [20]:
tf_idf = feature_extraction.text.TfidfVectorizer()